In [1]:
import cv2
import numpy as np

In [6]:
cap = cv2.VideoCapture(0)

ret,frame = cap.read()

face_cascade = cv2.CascadeClassifier("../DATA/haarcascades/haarcascade_frontalface_default.xml")
face_rects = face_cascade.detectMultiScale(frame)

face_x,face_y,w,h = tuple(face_rects[0])
track_win = (face_x,face_y,w,h)

roi = frame[face_y:face_y+h,face_x:face_x+w]

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

roi_hst = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_hst,roi_hst,0,255,cv2.NORM_MINMAX)

term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    
    ret,frame = cap.read()
    
    if ret==True:
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        dst = cv2.calcBackProject([hsv],[0],roi_hst,[0,180],1)
#         Meanshift method
#         ret,track_win = cv2.meanShift(dst,track_win,term_criteria)
#         x,y,w,h = track_win
#         img2 = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)

#         Continuously Adaptive Meanshift method
#         Not best to track faces - tracks other part of body that are moving as well
        ret,track_win = cv2.CamShift(dst,track_win,term_criteria)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        
        img2 = cv2.polylines(frame,[pts],True,(0,0,255),5)
        
        
        cv2.imshow("Face Tracking",img2)
        
        k=cv2.waitKey(1) & 0xFF
        
        if k==27:
            break;
            
cap.release()
cv2.destroyAllWindows()